In [1]:
import random
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
import numpy as np


In [2]:
import torch.utils.data as data
from PIL import Image
import os
import torch.nn as nn


class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, transform=None):
        self.root = data_root
        self.transform = transform

        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()

        self.n_data = len(data_list)

        self.img_paths = []
        self.img_labels = []

        for data in data_list:
            self.img_paths.append(data[:-3])
            self.img_labels.append(data[-2])

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

In [ ]:
# source_dataset_name = 'MNIST'
# target_dataset_name = 'mnist_m'
# # '/Users/purnavindhyakota/Downloads/DANN-master/dataset'
# source_image_root = os.path.join('/Users/purnavindhyakota/Downloads/DANN-master', 'dataset', source_dataset_name)
# target_image_root = os.path.join('/Users/purnavindhyakota/Downloads/DANN-master', 'content', target_dataset_name)
# model_root = os.path.join('..', 'models')
# cuda = True
# cudnn.benchmark = True
# lr = 1e-3
# batch_size = 128
# image_size = 28
# n_epoch = 100

# manual_seed = 2023
# random.seed(manual_seed)
# torch.manual_seed(manual_seed)

# # load data

# img_transform_source = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
# ])

# img_transform_target = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
# ])

# dataset_source = datasets.MNIST(
#     root='../dataset',
#     train=True,
#     transform=img_transform_source,
#     download=True
# )

# dataloader_source = torch.utils.data.DataLoader(
#     dataset=dataset_source,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=8)

# train_list = os.path.join(target_image_root, 'mnist_m_train_labels.txt')

# dataset_target = GetLoader(
#     data_root=os.path.join(target_image_root, 'mnist_m_train'),
#     data_list=train_list,
#     transform=img_transform_target
# )

# dataloader_target = torch.utils.data.DataLoader(
#     dataset=dataset_target,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=8)

##SVHN to MNIST

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# # Transform for SVHN (source)
# transform_svhn = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
# ])

# # Transform for MNIST (target)
# transform_mnist = transforms.Compose([
#     transforms.Resize((32, 32)),  # Resize MNIST images to 32x32 to match SVHN format
#     transforms.Grayscale(3),  # Convert MNIST images to 3 channels
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])
# batch_size=128


In [ ]:
# # Load SVHN dataset
# source_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
# dataloader_source = DataLoader(source_dataset, batch_size=batch_size, shuffle=True)

# # Load MNIST dataset
# mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
# dataloader_target = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)

Using downloaded and verified file: ./data/train_32x32.mat


100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100.0%
100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# for data in dataloader_target:
#   print(data[0].shape)
#   break

torch.Size([128, 3, 32, 32])


# source (syn digits), target (svhn)

In [5]:
# Transform for syn digits (source)
transform_syn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
])

# Transform for SVHN (target)
transform_svhn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
])

# # Transform for MNIST (target)
# transform_mnist = transforms.Compose([
#     transforms.Resize((32, 32)),  # Resize MNIST images to 32x32 to match SVHN format
#     transforms.Grayscale(3),  # Convert MNIST images to 3 channels
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])
batch_size=128

In [3]:
import os
import warnings

import torch
from PIL import Image
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive


class SyntheticDigits(VisionDataset):
    """Synthetic Digits Dataset.
    """

    resources = [
        ('https://github.com/liyxi/synthetic-digits/releases/download/data/synth_train.pt.gz',
         'd0e99daf379597e57448a89fc37ae5cf'),
        ('https://github.com/liyxi/synthetic-digits/releases/download/data/synth_test.pt.gz',
         '669d94c04d1c91552103e9aded0ee625')
    ]

    training_file = "synth_train.pt"
    test_file = "synth_test.pt"
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        """Init Synthetic Digits dataset."""
        super(SyntheticDigits, self).__init__(root, transform=transform, target_transform=target_transform)

        self.train = train

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError("Dataset not found." +
                               " You can use download=True to download it")

        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file

        print(os.path.join(self.processed_folder, data_file))

        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))

    def __getitem__(self, index):
        """Get images and target for data loader.
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], int(self.targets[index])

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.squeeze().numpy(), mode="RGB")

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """Return size of dataset."""
        return len(self.data)

    @property
    def raw_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder, self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder, self.test_file)))

    def download(self):
        """Download the Synthetic Digits data."""

        if self._check_exists():
            return

        os.makedirs(self.raw_folder, exist_ok=True)
        os.makedirs(self.processed_folder, exist_ok=True)

        # download files
        for url, md5 in self.resources:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.raw_folder,
                                         extract_root=self.processed_folder,
                                         filename=filename, md5=md5)

        print('Done!')

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load target SVHN dataset
target_svhn_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
dataloader_svhn_target = DataLoader(target_svhn_dataset, batch_size=batch_size, shuffle=True)

# Load source Syn Digits dataset
source_syn_dataset = SyntheticDigits(root='./syn_dataset', train=True, download=True, transform=transform_syn)
dataloader_syn_source = DataLoader(source_syn_dataset, batch_size=batch_size, shuffle=True)

Using downloaded and verified file: ./data/train_32x32.mat


100%|██████████| 892057743/892057743 [00:12<00:00, 72550753.86it/s]


Extracting ./syn_dataset/SyntheticDigits/raw/synth_train.pt.gz to ./syn_dataset/SyntheticDigits/processed


100%|██████████| 17783056/17783056 [00:00<00:00, 218212183.64it/s]


Extracting ./syn_dataset/SyntheticDigits/raw/synth_test.pt.gz to ./syn_dataset/SyntheticDigits/processed
Done!
./syn_dataset/SyntheticDigits/processed/synth_train.pt


In [8]:
for data in dataloader_svhn_target:
  print(data[0].shape)
  break
for data in dataloader_syn_source:
  print(data[0].shape)
  break

torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 32, 32])


# New Section

In [ ]:
# import torch.nn as nn

# from torch.autograd import Function


# class ReverseLayerF(Function):

#     @staticmethod
#     def forward(ctx, x, alpha):
#         ctx.alpha = alpha

#         return x.view_as(x)

#     @staticmethod
#     def backward(ctx, grad_output):
#         output = grad_output.neg() * ctx.alpha

#         return output, None

# class CNNModel(nn.Module):

#     def __init__(self):
#         super(CNNModel, self).__init__()
#         self.feature = nn.Sequential()
#         self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
#         self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
#         self.feature.add_module('f_pool1', nn.MaxPool2d(2))
#         self.feature.add_module('f_relu1', nn.ReLU(True))
#         self.feature.add_module('f_conv2', nn.Conv2d(64, 50, kernel_size=5))
#         self.feature.add_module('f_bn2', nn.BatchNorm2d(50))
#         self.feature.add_module('f_drop1', nn.Dropout2d())
#         self.feature.add_module('f_pool2', nn.MaxPool2d(2))
#         self.feature.add_module('f_relu2', nn.ReLU(True))

#         self.class_classifier = nn.Sequential()
#         self.class_classifier.add_module('c_fc1', nn.Linear(50 * 4 * 4, 100))
#         self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
#         self.class_classifier.add_module('c_relu1', nn.ReLU(True))
#         self.class_classifier.add_module('c_drop1', nn.Dropout2d())
#         self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
#         self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
#         self.class_classifier.add_module('c_relu2', nn.ReLU(True))
#         self.class_classifier.add_module('c_fc3', nn.Linear(100, 10))
#         self.class_classifier.add_module('c_softmax', nn.LogSoftmax())

#         self.domain_classifier = nn.Sequential()
#         self.domain_classifier.add_module('d_fc1', nn.Linear(50 * 4 * 4, 100))
#         self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
#         self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
#         self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
#         self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

#     def forward(self, input_data, alpha):
#         input_data = input_data.expand(input_data.data.shape[0], 3, 32,32)
#         feature = self.feature(input_data)
#         feature = feature.view(-1, 50 * 4 * 4)
#         reverse_feature = ReverseLayerF.apply(feature, alpha)
#         class_output = self.class_classifier(feature)
#         domain_output = self.domain_classifier(reverse_feature)

#         return class_output, domain_output

In [9]:
import torch.nn as nn
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

class CNNModel(nn.Module):
    def __init__(self, input_channels=3, num_classes=10, domain_classes=2):
        super(CNNModel, self).__init__()

        # Feature extractor
        self.feature = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Conv2d(64, 50, kernel_size=5),
            nn.BatchNorm2d(50),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(True)
        )

        # Class classifier
        self.class_classifier = nn.Sequential(
            nn.Linear(50 * 5 * 5, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Dropout2d(),
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, num_classes),
            nn.LogSoftmax(dim=1)
        )

        # Domain classifier
        self.domain_classifier = nn.Sequential(
            nn.Linear(50 * 5 * 5, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, domain_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, input_data, alpha):
      feature = self.feature(input_data)

    # Calculate the size of the flattened features
      size = feature.size()[1:]  # all dimensions except the batch dimension
      num_features = 1
      for s in size:
          num_features *= s

      feature = feature.view(-1, num_features)  # Automatically infer batch size
      reverse_feature = ReverseLayerF.apply(feature, alpha)
      class_output = self.class_classifier(feature)
      domain_output = self.domain_classifier(reverse_feature)

      return class_output, domain_output

In [13]:
import os
import torch
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ... (other necessary imports)

def test(dataset_name, epoch):
    assert dataset_name in ['MNIST', 'mnist_m', 'svhn', 'syn']

    model_root = '../syn_svhn_model_epoch.pth'
    image_root = '/Users/purnavindhyakota/Downloads/DANN-master/content/mnist_m'

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 32
    alpha = 0

    """ Load Data """
    img_transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])
    transform_mnist = transforms.Compose([
        transforms.Resize((32, 32)),  # Resize MNIST images to 32x32 to match SVHN format
        transforms.Grayscale(3),  # Convert MNIST images to 3 channels
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    if dataset_name == 'mnist_m':
        test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')
        dataset = GetLoader(
            data_root=os.path.join(image_root, 'mnist_m_test'),
            data_list=test_list,
            transform=img_transform
        )
    elif dataset_name == 'svhn':
        transform_svhn = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
        ])
        dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)
    elif dataset_name == 'syn':
        transform_syn = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
        ])
        dataset = SyntheticDigits(root='./syn_dataset', train=False, download=True, transform=transform_syn)

    else:  # MNIST
        dataset = datasets.MNIST(root='./data', train=False, transform=transform_mnist)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    """ Evaluate Model """
    my_net = torch.load(model_root)
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    n_total = 0
    n_correct = 0

    with torch.no_grad():
        for data in dataloader:
            t_img, t_label = data
            if cuda:
                t_img = t_img.cuda()
                t_label = t_label.cuda()

            class_output, _ = my_net(input_data=t_img, alpha=alpha)
            pred = class_output.data.max(1, keepdim=True)[1]
            n_correct += pred.eq(t_label.data.view_as(pred)).cpu().sum()
            n_total += t_label.size(0)

    accu = n_correct.item() * 1.0 / n_total
    print(f'epoch: {epoch}, accuracy of the {dataset_name} dataset: {accu}')


In [18]:
cuda = True
cudnn.benchmark = True
lr = 1e-3
batch_size = 128
image_size = 32
n_epoch = 100

manual_seed = 2023
random.seed(manual_seed)
torch.manual_seed(manual_seed)


my_net = CNNModel()
optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True



In [19]:
for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_svhn_target), len(dataloader_syn_source))
    data_source_iter = iter(dataloader_syn_source)
    data_target_iter = iter(dataloader_svhn_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = next(data_source_iter)
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)
        #print(batch_size)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)
        #print(input_img.shape)
        #print(class_label.shape)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        #print(class_output.shape)
        #print(domain_output.shape)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        data_target = next(data_target_iter)
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

    print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(), err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    torch.save(my_net, '../syn_svhn_model_epoch.pth')
    test('syn', epoch)
    test('svhn', epoch)



epoch: 0, [iter: 573 / all 573], err_s_label: 0.433821, err_s_domain: 0.573215, err_t_domain: 0.525964
./syn_dataset/SyntheticDigits/processed/synth_test.pt
epoch: 0, accuracy of the syn dataset: 0.9245263268083325
Using downloaded and verified file: ./data/test_32x32.mat
epoch: 0, accuracy of the svhn dataset: 0.758681622618316
epoch: 1, [iter: 573 / all 573], err_s_label: 0.261107, err_s_domain: 0.376089, err_t_domain: 0.629051
./syn_dataset/SyntheticDigits/processed/synth_test.pt
epoch: 1, accuracy of the syn dataset: 0.9455668376426254
Using downloaded and verified file: ./data/test_32x32.mat
epoch: 1, accuracy of the svhn dataset: 0.7973263675476336
epoch: 2, [iter: 573 / all 573], err_s_label: 0.389933, err_s_domain: 0.381912, err_t_domain: 0.213345
./syn_dataset/SyntheticDigits/processed/synth_test.pt
epoch: 2, accuracy of the syn dataset: 0.9532084162043337
Using downloaded and verified file: ./data/test_32x32.mat
epoch: 2, accuracy of the svhn dataset: 0.7808466502765826
epoch

In [16]:
my_net

CNNModel(
  (feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 50, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout2d(p=0.5, inplace=False)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ReLU(inplace=True)
  )
  (class_classifier): Sequential(
    (0): Linear(in_features=1250, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.5, inplace=False)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [17]:
def load_svhn(**kwargs):
    """Load SVHM dataloader.

    :**kwargs: arguments to pass to dataloader constructor
    :returns: train_loader, test_loader

    """
    transform = transforms.Compose(
        [transforms.Resize(28),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    trainset = torchvision.datasets.SVHN(root='./data',
                                         split='train',
                                         download=True,
                                         transform=transform)

    testset = torchvision.datasets.SVHN(root='./data',
                                        split='test',
                                        download=True,
                                        transform=transform)

    return get_loader(trainset, **kwargs),\
        get_loader(testset, **kwargs)